# Mexican forest cover Chatbot fine-tuning.

In this notebook, we will see how to fine-tune a custom chatbot (based on a Phi-4-mini model) using a hand-made training dataset.

This is a prompt-completion fine-tuning intended to generate SQL querys

Prerequisite: Create HuggingFace token with permission access to `defog/sqlcoder-7b-2`.

In [1]:
from datasets import Dataset, DatasetDict
import pandas as pd
from huggingface_hub import login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
from trl import SFTTrainer
from peft import LoraConfig, AutoPeftModelForCausalLM

Load the custom training dataset, available in this same github project.

In [2]:
excel_file_path = '/usr/workspace/media/training_prompts.xlsx'
df = pd.read_excel(excel_file_path)
hf_dataset = Dataset.from_pandas(df)
single_dataset_dict = DatasetDict({'train': hf_dataset})

In [3]:
single_dataset_dict['train'][0]

{'prompt': 'User request: ¿Cuál es el estado con mayor superficie cubierta por bosque?\n\nSQL:',
 'completion': 'SELECT\n  entidad_federativa,\n  superficie_cubierta_por_bosque\nFROM \n  superficie_bd.superficie_forestal\nORDER BY\n  superficie_cubierta_por_bosque\nDESC\nLIMIT 1;',
 'system_prompt': 'You are a SQL generator for ClickHouse database. Given a user request in natural language, you will respond with exactly one valid ClikHouse SQL query, nothing else. Use proper table and column names from the schema. Handle aggregations and filtering appropriately.',
 '__index_level_0__': 0}

Download LLM from HuggingFace and set up tokenizer. We'll use a 4bit quantization as I only have 8GB of Vram available.

In [ ]:
# Hugging Face login
my_token = "hf_xxxxxxx"
login(token=my_token)


# -------------------------------
# Load defog/sqlcoder-7b-2
# -------------------------------

model_id = 'defog/sqlcoder-7b-2'
tokenizer = AutoTokenizer.from_pretrained(model_id, token=my_token)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    token=my_token
)


# Make sure pad_token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Set up LoRA configurations, datasets and SFT (Supervised Fine-Tuning) training procedure.

In [5]:
# -------------------------------
# LoRA config
# -------------------------------

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
)

In [6]:
# ----------------------------------------
# Tokenization function (completion mode)
# ----------------------------------------

def tokenize_function(examples):
    prompts = examples["prompt"]
    completions = examples["completion"]
    texts = []
    for prompt, completion in zip(prompts, completions):
        # Concatenate prompt + completion
        text = prompt.strip() + " " + completion.strip()
        texts.append(text)
    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)


single_dataset_dict = single_dataset_dict.map(tokenize_function, batched=True)


Map:   0%|          | 0/202 [00:00<?, ? examples/s]

Start the fine-tuning with 150 training step, which will take ~2.5 hours on a GTX 4060 Laptop GPU with 8gb VRAM.

In [7]:
# -------------------------------
# Trainer
# -------------------------------

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=single_dataset_dict['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=150,
        learning_rate=2e-4,
        bf16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none",
    ),
    peft_config=lora_config,
)

trainer.train()

Truncating train dataset:   0%|          | 0/202 [00:00<?, ? examples/s]

Step,Training Loss
1,11.375000
2,11.223800
3,11.233600
4,8.461800
5,4.546200
6,2.797700
7,2.790500
8,2.485200
9,1.731100
10,2.086800


TrainOutput(global_step=150, training_loss=1.2262546783685684, metrics={'train_runtime': 9901.5406, 'train_samples_per_second': 0.061, 'train_steps_per_second': 0.015, 'total_flos': 1.211294350835712e+16, 'train_loss': 1.2262546783685684})

In [8]:
# Save only the LoRA adapter + tokenizer
trainer.model.save_pretrained("defog_sqlcoder_7b_2_f4bit")
tokenizer.save_pretrained("defog_sqlcoder_7b_2_f4bit")

('defog_sqlcoder_7b_2_f4bit/tokenizer_config.json',
 'defog_sqlcoder_7b_2_f4bit/special_tokens_map.json',
 'defog_sqlcoder_7b_2_f4bit/tokenizer.json')

### Test pipeline

In [1]:
from transformers import AutoTokenizer, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM
import torch

# -----------------------
# Quantization config (4-bit)
# -----------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # use bf16 on 4060
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# -----------------------
# Load tokenizer + model with LoRA adapter
# -----------------------
tokenizer = AutoTokenizer.from_pretrained("defog_sqlcoder_7b_2_f4bit")

model = AutoPeftModelForCausalLM.from_pretrained(
    "defog_sqlcoder_7b_2_f4bit",              # path to your LoRA adapter
    quantization_config=bnb_config, # load in 4-bit
    torch_dtype=torch.float16,
    device_map="auto"
)

torch.set_float32_matmul_precision('high')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
schema_info = """superficie_bd.superficie_forestal (
    entidad_federativa String,
    poblacion UInt32,
    superficie_entidad_federativa Float64,
    superficie_forestal Float64,
    superficie_no_forestal Float64,
    superficie_con_arbolado Float64,
    area_cubierta_por_bosque Float64,
    area_cubierta_por_selva Float64,
    area_cubierta_por_manglar Float64,
    superficie_cubierta_por_otras_areas_arboladas Float64,
    area_cubierta_por_matorral_xerofilo Float64,
    area_cubierta_por_otras_areas_forestales Float64,
    superficie_destinada_a_actividades_agricolas_de_humedad Float64,
    superficie_destinada_a_actividades_agricolas_de_riego Float64,
    superficie_destinada_a_actividades_agricolas_de_temporal Float64,
    superficie_de_cuerpos_de_agua Float64,
    superficie_destinada_a_actividades_acuicolas Float64,
    area_de_pastizales_cultivados Float64,
    area_de_pastizales_inducidos Float64,
    superficie_sin_vegetacion_visible Float64,
    superficie_desprovista_de_vegetacion Float64,
    superficie_ocupada_por_asentamientos_humanos Float64
)
"""

In [3]:
entidad_federativa = "['Aguascalientes', 'Baja California Norte', 'Baja California Sur', 'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua', 'Ciudad de Mexico', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Estado de Mexico', 'Michoacan', 'Morelos', 'Nayarit', 'Nuevo Leon', 'Oaxaca', 'Puebla', 'Queretaro', 'Quintana Roo', 'San Luis Potosi', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas']"

In [4]:
# -----------------------------
# MAIN FUNCTION
# -----------------------------
def nl_to_sql(request: str) -> str:
    prompt = f"""You are a SQL generator for PostgreSQL.
    Given a user request in natural language (in Spanish), respond with exactly one valid SQL query. Return ONLY the SQL query, no explanations, no markdown formatting.
    
    SCHEMA:
    Table: superficie_bd.superficie_forestal
    {schema_info}

    IMPORTANT RULES:

    1. GRANULARITY:
      - The table has one row per 'entidad_federativa' (state)
      - Available states: {entidad_federativa}

    2. TERMINOLOGY:
      - "entidad federativa" = "estado" (state)
      - cobertura = area = superficie
      - "México" (without qualifiers) = the entire country (aggregate all states)
      - "Estado de México" = a specific state
      - "Ciudad de México" = a specific state (different from México and Estado de México)
      - superficie total = sum of 'superficie_entidad_federativa'
      - "area forestal" is not the same as "area de bosque", these are different concepts (columns)

    3. WHEN TO AGGREGATE:
      - If user asks about "México" alone: Use SUM() without GROUP BY
      - If user asks about specific state(s): Use WHERE with exact state name(s)
      - If user asks about "all states" or "each state": Use GROUP BY entidad_federativa

    4. COLUMNS:
      - All columns except 'entidad_federativa' and 'poblacion' represent surface area in hectares
      - If user doesn't specify columns, select the most relevant based on context

    5. FORMATTING:
      - Always use: SELECT ... FROM superficie_bd.superficie_forestal
      - Use exact column and state names from the schema
      - For state filtering: WHERE entidad_federativa = 'exact_name'
      - For multiple states: WHERE entidad_federativa IN ('state1', 'state2')
      - Add ORDER BY when comparing or listing multiple results

    6. RULE EXAMPLES:
      - "superficie de México" → SUM(superficie_entidad_federativa) without WHERE
      - "superficie de Estado de México" → WHERE entidad_federativa = 'Estado de México'
      - "superficie forestal por estado" → SELECT entidad_federativa, superficie_forestal... GROUP BY entidad_federativa


    REQUEST EXAMPLE:
    User request: Dame la superficie de otras areas arboladas en la entidad federativa Sonora.
    SQL: SELECT
      superficie_cubierta_por_otras_areas_arboladas
    FROM
      superficie_bd.superficie_forestal
    WHERE
      entidad_federativa = 'Sonora';

    REQUEST EXAMPLE:
    User request: ¿Cuál es la entidad federativa con más superficie cubierta por bosque?
    SQL: SELECT
      entidad_federativa,
      superficie_cubierta_por_bosque
    FROM 
      superficie_bd.superficie_forestal
    ORDER BY
      superficie_cubierta_por_bosque
    DESC
    LIMIT 1;

    REQUEST EXAMPLE:
    User request: ¿Qué estado tiene la menor área de agua?
    SELECT
      entidad_federativa,
      superficie_cubierta_por_cuerpos_de_agua
    FROM 
      superficie_bd.superficie_forestal
    ORDER BY
      superficie_cubierta_por_cuerpos_de_agua
    ASC
    LIMIT 1;

    REQUEST EXAMPLE:
    User request: ¿Cuál es el total de superficie de matorral xerófilo?
    SQL: SELECT
      SUM(superficie_cubierta_por_matorral_xerofilo)
    FROM
      superficie_bd.superficie_forestal;

    REQUEST EXAMPLE:
    User request: ¿Cuál es la razón de superficie de manglar entre superficie total por entidad?
    SQL: SELECT
      entidad_federativa,
      superficie_cubierta_por_manglar/superficie_entidad_federativa
    FROM 
      superficie_bd.superficie_forestal
    ORDER BY
      superficie_cubierta_por_manglar/superficie_entidad_federativa
    DESC;

    REQUEST EXAMPLE:
    User request: ¿Cuál es la superficie cubierta por bosque en el Estado de México?
    SQL: SELECT
      superficie_cubierta_por_bosque
    FROM
      superficie_bd.superficie_forestal
    WHERE
      entidad_federativa = 'Estado de Mexico';

    REQUEST EXAMPLE:
    User request: ¿Cuánto ocupa la superficie destinada a asentamientos humanos?
    SQL: SELECT
      SUM(superficie_ocupada_por_asentamientos_humanos)
    FROM
      superficie_bd.superficie_forestal;

    REQUEST EXAMPLE:
    User request: ¿Cuál es la superficie forestal en la república?
    SQL: SELECT
      SUM(superficie_forestal)
    FROM
      superficie_bd.superficie_forestal;

    REQUEST EXAMPLE:
    User request: ¿Cuál es la razón de superficie cubierta por otras áreas arboladas entre superficie total en el país?
    SQL: SELECT
      SUM(superficie_cubierta_por_otras_areas_arboladas)/SUM(superficie_entidad_federativa)
    FROM 
      superficie_bd.superficie_forestal;

    Generate only the SQL query.

    User request: {request}

    SQL:
    """

    # 👇 Match input device to model’s first parameter device
    model_device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(model_device)

    gen_kwargs = {
        "max_new_tokens": 128,
        "do_sample": False,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id,
        #"stopping_criteria": stopping_criteria #conflicting line
    }

    @torch._dynamo.disable
    def safe_generate():
        with torch.no_grad():
            return model.generate(**inputs, **gen_kwargs)

    output_ids = safe_generate()
    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    sql = full_output.replace(prompt, "").strip()
    if ";" in sql:
      sql = sql.split(";")[0] + ";"
    
    return sql

In [ ]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    host='clickhouse',
    port=8123,  # HTTP interface
    username='abc',
    password='xyz'
)

In [6]:

query = nl_to_sql("Dame la superficie cubierta por agua en Jalisco")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT superficie_cubierta_por_cuerpos_de_agua FROM superficie_bd.superficie_forestal WHERE entidad_federativa = 'Jalisco';


In [7]:
result = client.query(query)

print(result.column_names)
result.result_rows

('superficie_cubierta_por_cuerpos_de_agua',)


[(161704.7867,)]

In [8]:
query = nl_to_sql("¿Cuál es la superficie cubierta por bosque en el Estado de México?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT superficie_cubierta_por_bosque FROM superficie_bd.superficie_forestal WHERE entidad_federativa = 'Estado de Mexico';


In [9]:
result = client.query(query)

print(result.column_names)
result.result_rows

('superficie_cubierta_por_bosque',)


[(623624.7159,)]

In [10]:
query = nl_to_sql("Muestra la razón de cuerpos de agua entre superficie total para Colima")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(superficie_cubierta_por_cuerpos_de_agua)/SUM(superficie_entidad_federativa) AS ratio FROM superficie_bd.superficie_forestal WHERE entidad_federativa = 'Colima';


In [11]:
result = client.query(query)

print(result.column_names)
result.result_rows

('ratio',)


[(0.016827571244275315,)]

In [12]:
query = nl_to_sql("¿Cuál es la razón del total de superficie cubierta por otras áreas forestales entre la población total?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(superficie_cubierta_por_otras_areas_arboladas)/NULLIF(SUM(poblacion), 0) AS ratio FROM superficie_bd.superficie_forestal;


In [13]:
result = client.query(query)

print(result.column_names)
result.result_rows

('ratio',)


[(0.004022882127771985,)]

In [14]:
query = nl_to_sql("¿Cuál es el estado con la mayor superficie destinada a actividades agrícolas de humedad?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT entidad_federativa, superficie_destinada_a_actividades_agricolas_de_humedad FROM superficie_bd.superficie_forestal ORDER BY superficie_destinada_a_actividades_agricolas_de_humedad DESC LIMIT 1;


In [15]:
result = client.query(query)

print(result.column_names)
result.result_rows

('entidad_federativa', 'superficie_destinada_a_actividades_agricolas_de_humedad')


[('Veracruz', 74343.30061)]

In [16]:
query = nl_to_sql("Área ocupada por asentamientos humanos")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(superficie_ocupada_por_asentamientos_humanos) AS superficie_ocupada_por_asentamientos_humanos FROM superficie_bd.superficie_forestal;


In [17]:
result = client.query(query)

print(result.column_names)
result.result_rows

('superficie_ocupada_por_asentamientos_humanos',)


[(2410991.0339599997,)]

In [18]:
query = nl_to_sql("¿Cuáles son las 3 entidades con mayor superficie sin vegetación visible?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT entidad_federativa, superficie_sin_vegetacion_visible FROM superficie_bd.superficie_forestal ORDER BY superficie_sin_vegetacion_visible DESC LIMIT 3;


In [19]:
result = client.query(query)

print(result.column_names)
result.result_rows

('entidad_federativa', 'superficie_sin_vegetacion_visible')


[('Baja California Norte', 304021.6308),
 ('Sonora', 136439.0437),
 ('Chihuahua', 93518.89663)]

In [20]:
query = nl_to_sql("¿Qué entidad federativa tiene la menor área de bosque?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT entidad_federativa, superficie_cubierta_por_bosque FROM superficie_bd.superficie_forestal ORDER BY superficie_cubierta_por_bosque ASC LIMIT 1;


In [21]:
result = client.query(query)

print(result.column_names)
result.result_rows

('entidad_federativa', 'superficie_cubierta_por_bosque')


[('Yucatan', 0.0)]

In [22]:
query = nl_to_sql("¿Cuál es la población total en México?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(poblacion) FROM superficie_bd.superficie_forestal;


In [23]:
result = client.query(query)

print(result.column_names)
result.result_rows

('SUM(poblacion)',)


[(131014024,)]

In [24]:
query = nl_to_sql("¿Cuál es el índice nacional de superficie de cuerpos de agua entre población?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(superficie_cubierta_por_cuerpos_de_agua)/NULLIF(SUM(poblacion), 0) AS indice_nacional_de_superficie_de_cuerpos_de_agua FROM superficie_bd.superficie_forestal;


In [25]:
result = client.query(query)

print(result.column_names)
result.result_rows

('indice_nacional_de_superficie_de_cuerpos_de_agua',)


[(0.02012530448456724,)]

In [26]:
query = nl_to_sql("¿Cuál es la entidad federativa con menor área de arbolado?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT entidad_federativa, MIN(superficie_cubierta_por_bosque) FROM superficie_bd.superficie_forestal GROUP BY entidad_federativa ORDER BY MIN(superficie_cubierta_por_bosque) ASC LIMIT 1;


In [27]:
result = client.query(query)

print(result.column_names)
result.result_rows

('entidad_federativa', 'MIN(superficie_cubierta_por_bosque)')


[('Quintana Roo', 0.0)]

In [30]:
query = nl_to_sql("¿Cuál es el área de Oaxaca?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT superficie_entidad_federativa FROM superficie_bd.superficie_forestal WHERE entidad_federativa = 'Oaxaca';


In [31]:
result = client.query(query)

print(result.column_names)
result.result_rows

('superficie_entidad_federativa',)


[(9376581.998,)]

In [34]:
query = nl_to_sql("Muestra la división de la suma de superficie forestal entre la suma de población")
print("Generated SQL:", sql)

Generated SQL: SELECT SUM(superficie_forestal)/NULLIF(SUM(poblacion), 0) AS division_superficie_poblacion FROM superficie_bd.superficie_forestal;


In [35]:
result = client.query(query)

print(result.column_names)
result.result_rows

('division_superficie_poblacion',)


[(1.0586288822053125,)]